# Nonlinear Fitting

What about the case of fitting to a function where the fit parameters enter in a nonlinear fashion?
For example:

$$f(x; a_0, a_1) = a_0 e^{a_1 x}$$

One trick that is often used for something like this is to transform the data.  So instead of fitting the data $(x_i, y_i)$, you instead fit $(x_i, \log y_i)$, and then our fitting function is:

$$\log f(x; a_0, a_1) = a_1 x + \log a_0$$

which is linear.

However, when there are errors associated with the $y_i$, the errors do not necessarily transform the correct way when you take the logarithm.  

So let's look at how we would fit directly to a nonlinear function.